# BreakItDown Tasks

### 1. Mark video as watched
- If ID exist
- query result skip over id and check again if ID exists
- repeat until not 

### 2. Revised percentage calculation (relative instead of 100%)
Will cause a tag to be dominant tho (weight 1000 while others still 10). Mitigated with intentional interruptions (#4). 

### 3. User class schema
- self
- interest_weights (dict)
- watched_videos (dict)

### 4. Intentional Interruptions
Handle dominations
- 50% -> Suggest relevant interests 
- 75% -> "Have you been enjoying {tag name} so far? "yes". "recommend more". 

### 5. Modify whisper prompt to return most relevant tag first then extract that for on_user_swipe()
in [github](https://github.com/edgurinc/edgur/blob/main/backend/accounts/videoprocessors.py)
- To help us later in handling which tag to add weights to given videos have multiple tags. 

### 6. Decay other interests whenever watching a video
except current video tag 

### 7. Handle irrelevant tags
Acknowledge that it will be filled soon, such as "food" category

### 8. "Cache" next 4 videos
Run uniformDist+reco only on 4th video 

### 9. Min selection Max selection for categories
Like tiktok's

In [3]:
from pprint import pprint
import random

In [ ]:
# Constants

new_video_weight = 5
queued_videos_count = 5 # Number of vids to queue before running query again (to save costs) AKA n_results = 5


# Simulation Only (replace user input)
chosen_tag = "Microwave"


In [4]:
def create_user_interest_dict(*interests):
    interest_count = len(interests)
    interest_weight = (100 - 10) / interest_count  # Subtract 10 for 'Random'
    user_interests = {interest: interest_weight for interest in interests}
    user_interests["Random"] = 10  # Add 'Random' with constant value of 10

    print(
        f"Interest weights: {sum(user_interests.values())}"
    )  # Sum of all weights should be 100

    return user_interests

In [9]:
# New user selects interests (max 5)
# 10 total - 5 start to discover other 5 naturally

carl_interests = create_user_interest_dict(
    "DevOps",
    "Tests (SAT)",
    "Machine Learning",
    "Cats",
    "Cooking",
    "LEL"
)

pprint(carl_interests)

Interest weights: 100.0
{'Cats': 15.0,
 'Cooking': 15.0,
 'DevOps': 15.0,
 'LEL': 15.0,
 'Machine Learning': 15.0,
 'Random': 10,
 'Tests (SAT)': 15.0}


In [ ]:
# HELPER FUNCTIONS for update_weight


def observe_action_taken(video_tag, liked, shared, watched, loop_count):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    total_points = 0

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points


def handle_update_weight(user_dict_interest, video_tag, total_points):
    """
    This function updates the user's interest score based on the total points.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.
    total_points (float): The total points calculated based on the user's actions.

    Returns:
    dict: The updated user's interest dictionary.
    """
    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    # Deduct the total points proportionally from existing interests (excluding 'Random' and the video_tag itself)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(
        f"Total weight to deduct from other interests: {total_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}"
    )

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (total_points / total_deductable_weight) * user_dict_interest[
                interest
            ]
            user_dict_interest[interest] -= deduction

            print(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest


def handle_add_new_tag(user_dict_interest, video_tag):
    """
    This function adds a new interest to the user's interest dictionary.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.

    Returns:
    dict: The updated user's interest dictionary with the new interest added.
    """
    # Add new interest if it doesn't exist and give initial points
    initial_points = 5
    user_dict_interest[video_tag] = initial_points
    print(f"Added new interest {video_tag} with initial weight: {initial_points}")

    # Deduct initial points proportionally from existing interests (excluding 'Random' and current video_tag)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(
        f"Total weight to deduct from other interests: {initial_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}\n"
    )

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (initial_points / total_deductable_weight) * user_dict_interest[
                interest
            ]
            user_dict_interest[interest] -= deduction

            pprint(
                f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}"
            )

    return user_dict_interest